In [ ]:
import torch
import torch.nn as nn

from lib.architecture import Search
from lib.sample import SampleNormal

import math

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device

In [ ]:
N = 1000  # num_samples_per_class
D = 2  # dimensions
C = 3  # num_classes

X = torch.zeros(N * C, D).to(device)
y = torch.zeros(N * C, dtype=torch.long).to(device)

for c in range(C):
    index = 0
    t = torch.linspace(0, 1, N)
    # When c = 0 and t = 0: start of linspace
    # When c = 0 and t = 1: end of linpace
    # This inner_var is for the formula inside sin() and cos() like sin(inner_var) and cos(inner_Var)
    inner_var = torch.linspace(
        # When t = 0
        (2 * math.pi / C) * (c),
        # When t = 1
        (2 * math.pi / C) * (2 + c),
        N
    ) + torch.randn(N) * 0.2
    
    for ix in range(N * c, N * (c + 1)):
        X[ix] = t[index] * torch.FloatTensor((
            math.sin(inner_var[index]), math.cos(inner_var[index])
        ))
        y[ix] = c
        index += 1

print("Shapes:")
print("X:", tuple(X.size()))
print("y:", tuple(y.size()))

In [ ]:
D = 2
H = 5
O = 3

normalize = nn.LayerNorm(H)

encoder = nn.Sequential(
    nn.Linear(D, H),
    nn.ReLU(),
    normalize
)

search = Search(
    transition=nn.Sequential(
        nn.Linear(H, 2*H),
        nn.ReLU()
    ),
    fitness=nn.Sequential(
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, 1),
    ),
    sample=nn.Sequential(
        SampleNormal(H, num_samples=8),
        normalize
    ),
    max_depth=2,
    beam_width=8
)

decoder = nn.Sequential(
    nn.Linear(H, 3)
)

model = nn.Sequential(
    encoder,
    search,
    decoder
)

model.to(device)

In [ ]:
learning_rate = 1e-3
lambda_l2 = 1e-5

In [ ]:

# nn package also has different loss functions.
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss()

# we use the optim package to apply
# ADAM for our parameter updates
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# e = 1.  # plotting purpose
from torch.utils.data import TensorDataset, DataLoader

with device:
    dataset = TensorDataset(X, y)
    data_loader = DataLoader(dataset, 100, shuffle=True, generator=torch.Generator(device))

    # Training
    for t in range(100):
        for batch, targets in data_loader:

            # Feed forward to get the logits
            y_pred = model(batch)
            
            # loss
            loss = criterion(y_pred, targets)
            
            # accuracy
            score, predicted = torch.max(y_pred, 1)
            acc = (targets == predicted).sum().float() / len(targets)
            
            print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
            # display.clear_output(wait=False)
            
            # zero the gradients before running
            # the backward pass.
            optimizer.zero_grad()

            # clip gradient
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1e-2)
            
            # Backward pass to compute the gradient
            # of loss w.r.t our learnable params. 
            loss.backward()
            
            # Update params
            optimizer.step()